In [3]:
def compute_area_between_curves(morf_curve, lerf_curve):
    """
    Compute the area between the LeRF and MoRF curves using the trapezoidal rule. In the literature, we divide
    the area by the number of steps to normalize it.
    """
    # Ensure the curves are of the same length
    assert len(morf_curve) == len(lerf_curve), "Curves must be of the same length"

    # Compute the area using the trapezoidal rule
    area = 0.0
    for i in range(1, len(morf_curve)):
        area += 0.5 * ((lerf_curve[i] - morf_curve[i]) + (lerf_curve[i-1] - morf_curve[i-1]))
    return area / len(morf_curve)


def compute_aopc(morf_curve):
    # Compute the Area Over the Perturbation Curve (AOPC)
    reference = morf_curve[0]

    area = 0.0
    for i in range(1, len(morf_curve)):
        area += 0.5 * ((reference - morf_curve[i]) + (reference - morf_curve[i-1]))
    return area / len(morf_curve)

def normalized_abpc(morf_curve, lerf_curve):
    abpc = compute_area_between_curves(morf_curve, lerf_curve)
    range = max(lerf_curve) - min(morf_curve)
    return abpc / range

def normalized_aopc(morf_curve):
    aopc = compute_aopc(morf_curve)
    range = max(morf_curve) - min(morf_curve)
    return aopc / range


In [6]:
# patch_fcc_fp_fix_with_sv_axis_debug.py
# Fix FCC-organs false_positive_aggregation runs in group "ABPC-volumes":
# - Curves were swapped: use corrected (MoRF <- LeRF, LeRF <- MoRF)
# - ABPC sign/definition fixed via recomputation from corrected curves (and /N normalization handled by your functions if applicable)
# - AOPC recomputed from corrected MoRF; normalized metrics recomputed
# - Print OLD vs NEW metrics; by default DRY_RUN=True (no writes)

import os
import numpy as np
import wandb
from typing import Dict, List, Optional

# ===== USER CONFIG =====
ENTITY = "giuliosichili"
PROJECT = "automi"
GROUP   = "ABPC-volumes"

TARGET_SV_TYPE = "FCC-organs"
TARGET_AGG     = "false_positive_aggregation"

X_KEY    = "supervoxels_perturbed"   # your auxiliary x-axis
PAGE_SIZE = 512

DRY_RUN  = True   # << SAFE DEFAULT: ONLY PRINT, NO WRITES >>
VERBOSE  = True   # << PRINT per-run details >>
# =======================

# Externally provided functions must exist in your environment:
#   compute_area_between_curves(morf_curve, lerf_curve)
#   compute_aopc(morf_curve)
#   normalized_abpc(morf_curve, lerf_curve)
#   normalized_aopc(morf_curve)

def _fmt8(x: Optional[float]) -> str:
    if x is None:
        return "None"
    try:
        xf = float(x)
        if np.isnan(xf):
            return "nan"
        return f"{xf:.8f}"
    except Exception:
        return str(x)

def _fmt4(x: Optional[float]) -> str:
    if x is None:
        return "None"
    try:
        xf = float(x)
        if np.isnan(xf):
            return "nan"
        return f"{xf:.4f}"
    except Exception:
        return str(x)

def fetch_target_runs(api: wandb.Api) -> List[wandb.apis.public.Run]:
    """Find only FCC-organs + false_positive_aggregation runs inside GROUP."""
    path = f"{ENTITY}/{PROJECT}"
    filt = {
        "config.group": {"$eq": GROUP},
        "config.supervoxel_type": {"$eq": TARGET_SV_TYPE},
        "config.aggregation_function": {"$eq": TARGET_AGG},
    }
    return list(api.runs(path, filters=filt))

def collect_curves_with_x(run: wandb.apis.public.Run):
    """Collect MoRF/LeRF aligned by the custom x-axis (supervoxels_perturbed); fallback to step if missing."""
    morf_map, lerf_map = {}, {}
    x_seen = set()
    for row in run.scan_history(page_size=PAGE_SIZE):
        step = row.get("_step", row.get("step"))
        if step is None:
            continue
        # x value for plotting/aligning
        x = row.get(X_KEY, None)
        if x is None:
            x = step  # fallback to step if not logged
        try:
            x = int(x)
        except Exception:
            try:
                x = int(float(x))
            except Exception:
                continue
        x_seen.add(x)

        m = row.get("MoRF", None)
        l = row.get("LeRF", None)
        if m is not None:
            try: morf_map[x] = float(m)
            except Exception: pass
        if l is not None:
            try: lerf_map[x] = float(l)
            except Exception: pass

    common_x = sorted(set(morf_map.keys()) & set(lerf_map.keys()))
    if not common_x:
        return np.array([]), np.array([]), np.array([])

    morf = np.array([morf_map[i] for i in common_x], dtype=float)
    lerf = np.array([lerf_map[i] for i in common_x], dtype=float)
    xs   = np.array(common_x, dtype=int)
    return morf, lerf, xs

def recompute_from_corrected(morf_fixed: np.ndarray, lerf_fixed: np.ndarray) -> Dict[str, float]:
    """Compute corrected metrics from corrected curves using user's functions."""
    out: Dict[str, float] = {}
    # ABPC (assumes your compute_area_between_curves implements the correct formulation)
    out["ABPC_area"] = float(compute_area_between_curves(morf_fixed, lerf_fixed))
    # AOPC from MoRF only
    out["AOPC"] = float(compute_aopc(morf_fixed))
    # normalized variants
    out["norm_ABPC"] = float(normalized_abpc(morf_fixed, lerf_fixed))
    out["norm_AOPC"] = float(normalized_aopc(morf_fixed))
    return out

def old_summary_metrics(run: wandb.apis.public.Run) -> Dict[str, Optional[float]]:
    s = run.summary or {}
    def g(k):
        v = s.get(k)
        try:
            return float(v) if v is not None else None
        except Exception:
            return None
    return {
        "ABPC_area": g("ABPC_area"),
        "AOPC": g("AOPC"),
        "norm_ABPC": g("norm_ABPC"),
        "norm_AOPC": g("norm_AOPC"),
    }

def main():
    api = wandb.Api()
    runs = fetch_target_runs(api)
    print(f"[target] group='{GROUP}', sv_type='{TARGET_SV_TYPE}', agg='{TARGET_AGG}': {len(runs)} runs")

    patched, skipped = 0, 0
    for run in runs:
        try:
            morf, lerf, xs = collect_curves_with_x(run)
            if morf.size == 0 or lerf.size == 0:
                if VERBOSE:
                    print(f"- {run.id} SKIP: no aligned MoRF/LeRF on '{X_KEY}'")
                skipped += 1
                continue

            # Correction: swap MoRF/LeRF
            morf_fixed = lerf.copy()
            lerf_fixed = morf.copy()

            # Recompute corrected metrics
            new_metrics = recompute_from_corrected(morf_fixed, lerf_fixed)
            old_metrics = old_summary_metrics(run)

            # Pretty print OLD vs NEW
            if VERBOSE:
                print(f"\nRun {run.id} | N={len(xs)} | x in [{xs.min()}, {xs.max()}]")
                print("  ABPC_area : old =", _fmt8(old_metrics["ABPC_area"]), " -> new =", _fmt8(new_metrics["ABPC_area"]))
                print("  AOPC      : old =", _fmt8(old_metrics["AOPC"]),      " -> new =", _fmt8(new_metrics["AOPC"]))
                print("  norm_ABPC : old =", _fmt4(old_metrics["norm_ABPC"]), " -> new =", _fmt4(new_metrics["norm_ABPC"]))
                print("  norm_AOPC : old =", _fmt4(old_metrics["norm_AOPC"]), " -> new =", _fmt4(new_metrics["norm_AOPC"]))

            if DRY_RUN:
                patched += 1
                continue

            # ---- WRITE (summary only; silent) ----
            # Overwrite summary metrics with corrected values
            updates = dict(new_metrics)
            # Keep a small breadcrumb if you like (prev values)
            updates.update({
                "ABPC_area_prev": old_metrics["ABPC_area"],
                "AOPC_prev": old_metrics["AOPC"],
                "norm_ABPC_prev": old_metrics["norm_ABPC"],
                "norm_AOPC_prev": old_metrics["norm_AOPC"],
            })
            run.summary.update(updates, overwrite=True)

            patched += 1

        except Exception as e:
            if VERBOSE:
                print(f"- {run.id} ERROR: {e}")
            skipped += 1

    print(f"\nDone. Candidate runs: {len(runs)} | Patched (or would patch in DRY_RUN): {patched} | Skipped: {skipped} | "
          f"DRY_RUN={DRY_RUN} | VERBOSE={VERBOSE}")

if __name__ == "__main__":
    main()

[target] group='ABPC-volumes', sv_type='FCC-organs', agg='false_positive_aggregation': 6 runs

Run yuo4p6ll | N=428 | x in [0, 427]
  ABPC_area : old = -0.00022985  -> new = 0.00022985
  AOPC      : old = 0.00001456  -> new = 0.00024441
  norm_ABPC : old = -0.9377  -> new = 0.8218
  norm_AOPC : old = 0.0594  -> new = 0.8738

Run ci5gxcdn | N=368 | x in [0, 367]
  ABPC_area : old = -0.00039432  -> new = 0.00039432
  AOPC      : old = 0.00003385  -> new = 0.00042817
  norm_ABPC : old = -1.5355  -> new = 0.7253
  norm_AOPC : old = 0.1318  -> new = 0.7876

Run mherfqjc | N=339 | x in [0, 338]
  ABPC_area : old = -0.00042865  -> new = 0.00042865
  AOPC      : old = 0.00004021  -> new = 0.00046886
  norm_ABPC : old = -2.5781  -> new = 0.7793
  norm_AOPC : old = 0.2418  -> new = 0.8524

Run w7mwddqi | N=416 | x in [0, 415]
  ABPC_area : old = -0.00011613  -> new = 0.00011613
  AOPC      : old = 0.00007934  -> new = 0.00019548
  norm_ABPC : old = -0.9556  -> new = 0.4636
  norm_AOPC : old = 0.